In [2]:
## try adding memory to these agents 

import random 
import numpy as np 
import gym
import torch
import torch.nn as nn
import torch.optim as optim 
from lanczos import l_lanczos, combine_krylov_spaces 

class Object(object):
    pass

class ReplayBuffer(): 
    def __init__(self, capacity=10000): 
        self.n = 0 
        self.capacity = capacity 
        self.state_list = [] 
        self.action_list = [] 
        self.reward_list = [] 
        self.next_state_list = [] 
        self.done_list = [] 
        pass 
    def __len__(self):
         return self.n 
    def add(self, state, action, reward, next_state, done): 
        if self.n >= self.capacity: 
            ## discard earliest observation 
            self.state_list = self.state_list[1:] 
            self.action_list = self.action_list[1:] 
            self.reward_list = self.reward_list[1:] 
            self.next_state_list = self.next_state_list[1:] 
            self.done_list = self.done_list[1:] 
            self.n -= 1 
        pass 
        ## cast to torch  
        state = torch.tensor(state) 
        action = torch.tensor(action) 
        reward = torch.tensor(reward) 
        next_state = torch.tensor(next_state) 
        done = torch.tensor(done) 
        ## append to buffer 
        self.state_list.append(state) 
        self.action_list.append(action) 
        self.reward_list.append(reward) 
        self.next_state_list.append(next_state) 
        self.done_list.append(done) 
        self.n += 1 
        pass 
    def sample(self, batch_size=32, idx_list=None): 
        ## sample lists 
        out = Object() ## transitions 
        out.state = [] 
        out.action = [] 
        out.reward = [] 
        out.next_state = [] 
        out.done = [] 
        if idx_list is not None: 
            batch_size = len(idx_list) 
        for i in range(batch_size): 
            if idx_list is None: 
                idx = random.randint(0, self.n-1) 
            else: 
                idx = idx_list[i]
                pass 
            out.state.append(self.state_list[idx]) 
            out.action.append(self.action_list[idx]) 
            out.reward.append(self.reward_list[idx]) 
            out.next_state.append(self.next_state_list[idx]) 
            out.done.append(self.done_list[idx]) 
            pass 
        ## stack  
        out.state = torch.stack(out.state) 
        out.action = torch.stack(out.action) 
        out.reward = torch.stack(out.reward).reshape([-1,1]) 
        out.next_state = torch.stack(out.next_state) 
        out.done = torch.stack(out.done).reshape([-1,1]) 
        return out 
    def clear(self, n=None): 
        'clears first `n` transitions, or all if `n is None`'
        if n is None: 
            n = self.n 
            pass 
        self.state_list = self.state_list[n:] 
        self.action_list = self.action_list[n:] 
        self.reward_list = self.reward_list[n:] 
        self.next_state_list = self.next_state_list[n:] 
        self.done_list = self.done_list[n:] 
        self.n = len(self.state_list) 
        pass 
    pass 

# Define the actor and critic networks 
class SSRAgent(nn.Module): 
    def __init__(self, replay_buffer, ssr_rank=2): 
        super(SSRAgent, self).__init__() 
        self.ssr_rank = ssr_rank 
        self.ssr_low_rank_matrix = None ## =: A 
        self.ssr_residual_diagonal = None ## =: resid 
        self.ssr_center = None 
        self.ssr_n = None 
        ## N * Fisher Information \approx AA^T + resid 
        self.ssr_model_dimension = None 
        self.replay_buffer = replay_buffer 
        pass 
    def loss(self, transitions): 
        raise NotImplementedError('ERROR: loss not implemented!') 
    def memorize(self, n=None): 
        'memorize oldest `n` transitions, or all if `n is None`' 
        self.ssr_center = self.__get_param() ## elliptical centroid 
        if self.ssr_model_dimension is None: 
            self.ssr_model_dimension = self.ssr_center.shape[0] 
            pass 
        ssr_low_rank_matrix, ssr_residual_diagonal = l_lanczos(self.__get_grad_generator(n), self.ssr_rank, self.ssr_model_dimension, calc_diag=True) 
        if self.ssr_low_rank_matrix is None: 
            ## first memorization 
            self.ssr_low_rank_matrix = ssr_low_rank_matrix 
            self.ssr_residual_diagonal = ssr_residual_diagonal 
            self.ssr_n = n 
        else: 
            ## combine with previous memories 
            self.ssr_low_rank_matrix = combine_krylov_spaces(self.ssr_low_rank_matrix, ssr_low_rank_matrix) 
            self.ssr_residual_diagonal += ssr_residual_diagonal 
            self.ssr_n += n 
            pass 
        pass 
    def ssr(self, lmbda=None): 
        'get the ssr regularizer'
        if self.ssr_low_rank_matrix is None: 
            return 0. 
        if lmbda is None: 
            lmbda = 1. 
            pass 
        p = self.__get_param() 
        p0 = self.ssr_center 
        d = p - p0 
        A = self.ssr_low_rank_matrix 
        res = self.ssr_residual_diagonal 
        dA = d.matmul(A) 
        dAT = dA.transpose(0,1) 
        dresdT = d.matmul(res).matmul(d.transpose(0,1))
        ssr_sum = dA.matmul(dAT) + dresdT 
        if lmbda is None: 
            return .5 * ssr_sum ## natural value ## TODO adjust losses away from means to use this 
        return lmbda * .5 * ssr_sum / self.ssr_n ## adjusted sample size 
    def __get_param(self): 
        return torch.cat([p.reshape([-1, 1]) for p in self.parameters()], dim=0) 
    def __get_get_grad_generator(self, n=None): 
        ## The double get hides `self` in a function context,  
        ## packaging `get_grad_generator` for calling without 
        ## the SSRAgent instance.  
        if n is None: 
            n = self.replay_buffer.n 
            pass 
        def get_grad_generator(): 
            'l-Lanczos alg uses grad at least `ssr_rank` times'
            def grad_generator(): 
                self.eval() 
                for idx in range(n): 
                    transition = self.replay_buffer.sample(idx_list=[idx]) 
                    loss = self.loss(transition) 
                    loss.backward() 
                    grad_vec = torch.cat([p.grad.reshape([-1]) for p in self.parameters()]) 
                    yield grad_vec 
                    pass
                pass 
            return grad_generator 
        pass 
    pass 

class Actor(SSRAgent):
    def __init__(self, state_dim, action_dim, critic=None, replay_buffer=None, ssr_rank=2): 
        super(Actor, self).__init__(replay_buffer=replay_buffer, ssr_rank=ssr_rank) 
        self.fc1 = nn.Linear(state_dim, 256) 
        self.fc2 = nn.Linear(256, 128) 
        self.fc3 = nn.Linear(128, action_dim) 
        self.critic = critic 
        pass 
    def forward(self, state): 
        x = torch.relu(self.fc1(state)) 
        x = torch.relu(self.fc2(x)) 
        action = torch.tanh(self.fc3(x)) 
        return action
    def loss(self, transitions): 
        if self.critic is None: 
            raise ValueError('ERROR: this model has no associated critic!') 
        return -torch.mean(self.critic(transitions.state, self(transitions.state))) 
    pass 

class Critic(SSRAgent):
    def __init__(self, state_dim, action_dim, target_actor=None, target_critic=None, replay_buffer=None, ssr_rank=2): 
        super(Critic, self).__init__(replay_buffer=replay_buffer, ssr_rank=ssr_rank) 
        self.fc1 = nn.Linear(state_dim + action_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.target_actor = target_actor 
        self.target_critic = target_critic 
        pass 
    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        value = self.fc3(x)
        return value
    def loss(self, transitions): 
        if target_actor is None or target_critic is None: 
            raise ValueError('ERROR: target model missing!') 
        # Calculate the target Q-values
        target_Q = self.target_critic(transitions.next_state, self.target_actor(transitions.next_state))
        target_Q = (1 - transitions.done.int()) * target_Q.clone().detach() * GAMMA + transitions.reward 
        # Calculate the current Q-values
        current_Q = self(transitions.state, transitions.action) 
        # Calculate the critic loss
        return torch.mean((target_Q - current_Q).pow(2))
    pass 

# Create the replay buffer 
replay_buffer = ReplayBuffer(capacity=100000) 

# Create the actor and critic networks
target_critic = Critic(state_dim=4, action_dim=1) 
target_actor = Actor(state_dim=4, action_dim=1) 
critic = Critic(state_dim=4, action_dim=1, target_critic=target_critic, target_actor=target_actor) 
actor = Actor(state_dim=4, action_dim=1, critic=critic) 

# Define the optimizers
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-5)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

# Define the environment 
env = gym.make('CartPole-v1') 

reward_list = [] 
p_list = [] # DEBUG 
GAMMA = 0.99 
LAMBDA = 1. 

# Train the agent
for episode in range(1000):
    state = env.reset()
    target_actor.load_state_dict(actor.state_dict())
    target_critic.load_state_dict(critic.state_dict())

    for t in range(1000):
        action = actor(torch.tensor(state)) 
        if np.random.binomial(1, max(0,50-episode)/50) > 0: 
            ## random action
            action = torch.tensor(np.random.uniform(low=-1., high=1.)).reshape([1]) 
            pass 
        
        action_p = action.item() * .5 + .5 
        action_int = np.random.binomial(1, action_p) ## must be 0 or 1 
        next_state, reward, done, _ = env.step(action_int) 

        replay_buffer.add(state, action, reward, next_state, done) 

        if len(replay_buffer) > 1000: 
            # Sample a batch of transitions from the replay buffer 
            transitions = replay_buffer.sample(batch_size=256) 

            # Calculate the critic loss 
            critic_loss = critic.loss(transitions) + critic.ssr(LAMBDA) 

            # Update the critic network 
            critic_optimizer.zero_grad() 
            critic_loss.backward() 
            critic_optimizer.step() 
            
            # Calculate the actor loss 
            actor_loss = actor.loss(transitions) + actor.ssr(LAMBDA) 

            # Update the actor network 
            actor_optimizer.zero_grad() 
            actor_loss.backward() 
            actor_optimizer.step() 

            if len(replay_buffer) > 2000: 
                ## memoryize every 1000 transitions 
                actor.memorize(1000) 
                critic.memorize(1000) 
                replay_buffer.clear(1000)  
                pass 
            pass 

        state = next_state 

        if done:
            break

    # Evaluate the agent
    episode_reward = 0 
    state = env.reset() 

    for t in range(1000): 
        action = actor(torch.tensor(state))

        action_p = action.item() * .5 + .5 
        action_int = np.random.binomial(1, action_p) 
        next_state, reward, done, _ = env.step(action_int) 
        p_list.append(action.item()) # DEBUG 

        episode_reward += reward
        state = next_state 

        if done:
            break
    
    reward_list.append(episode_reward)
    print(f'Episode {episode}: {episode_reward}, len(replay_buffer): {len(replay_buffer)}')

/tmp/ipykernel_4190/468116197.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action = torch.tensor(action)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Episode 0: 29.0, len(replay_buffer): 44
Episode 1: 26.0, len(replay_buffer): 61
Episode 2: 12.0, len(replay_buffer): 77
Episode 3: 24.0, len(replay_buffer): 109
Episode 4: 18.0, len(replay_buffer): 123
Episode 5: 13.0, len(replay_buffer): 147
Episode 6: 12.0, len(replay_buffer): 160
Episode 7: 22.0, len(replay_buffer): 191
Episode 8: 15.0, len(replay_buffer): 221
Episode 9: 10.0, len(replay_buffer): 241
Episode 10: 28.0, len(replay_buffer): 255
Episode 11: 19.0, len(replay_buffer): 275
Episode 12: 13.0, len(replay_buffer): 307
Episode 13: 21.0, len(replay_buffer): 349
Episode 14: 12.0, len(replay_buffer): 398
Episode 15: 12.0, len(replay_buffer): 408
Episode 16: 22.0, len(replay_buffer): 425
Episode 17: 20.0, len(replay_buffer): 448
Episode 18: 27.0, len(replay_buffer): 461
Episode 19: 13.0, len(replay_buffer): 475
Episode 20: 16.0, len(replay_buffer): 489
Episode 21: 25.0, len(replay_buffer): 498
Episode 22: 15.0, len(replay_buffer): 519
Episode 23: 11.0, len(replay_buffer): 536
Episo

KeyboardInterrupt: 